In [1]:
import tensorflow as tf
import scipy.io as scio
import scipy.sparse as sp
import pandas as pd
import numpy as np

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [4]:
from tensorflow.keras import activations, regularizers, constraints, initializers

class GCNConv(tf.keras.layers.Layer):
    def __init__(self,
                 units,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(GCNConv, self).__init__()

        self.units = units
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
    
  


    def build(self, input_shape):

        """ GCN has two inputs : [An, X]
        """

        fdim = input_shape[1][-1]  # feature dim,取特征长度
        # 初始化权重矩阵
    
        
        self.weight=self.add_weight(name="weight",
                               shape=(fdim, self.units),
                               initializer=self.kernel_initializer,
                               trainable=True)
       
            
        self.bias = self.add_weight(name="bias",
                               shape=(self.units ),
                               initializer=self.bias_initializer,
                               trainable=True)
      
            
            

    def call(self, inputs):
        """ GCN has two inputs : [An, X]
        """
        #读入的是一个邻接矩阵
        self.An=inputs[0]
        self.X = inputs[1]
        

        # 计算 XW 
        #sparse_matmul
        
        h = tf.add(tf.matmul(self.X, self.weight),self.bias) 
            
        # 计算 AXW ,A矩阵已经处理过的
        
        output = tf.sparse.sparse_dense_matmul(self.An, h)

        if self.activation:
            output = self.activation(output)

        return output

In [5]:
import tensorflow as tf
from tensorflow.keras import activations, regularizers, constraints, initializers

class GCMC_Conv(tf.keras.layers.Layer):
    def __init__(self,
                 units,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(GCMC_Conv, self).__init__()

        self.units = units #要降成的维度
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
    
  


    def build(self, input_shape):
        #print(input_shape)

        """
        GC-MC has three inputs : [An,X，Z]
        X is self_feature,Z is another node type feature
        
        """
        
        
        ###   目标为XW1+(CA)ZW2    ###
        
        
         
        ###   这里开始为设置XW1的参数
        
        
        fdim_X = input_shape[1][-1] # feature dim,取X特征长度

        
        self.weight_1=self.add_weight(name="weight_1",
                               shape=(fdim_X, self.units),
                               initializer=self.kernel_initializer,
                               trainable=True)
       
        self.bias_1 = self.add_weight(name="bias_1",
                               shape=(self.units ),
                               initializer=self.bias_initializer,
                               trainable=True)
        
        ###   这里设置(CA)ZW2的参数
        
        fdim_Z=input_shape[2][-1]
        
        self.weight_2=self.add_weight(name="weight_2",
                               shape=(fdim_Z, self.units), #fdim_Z这个不对，不是12100，是5403
                               initializer=self.kernel_initializer,
                               trainable=True)
       
        self.bias_2 = self.add_weight(name="bias_2",
                               shape=(self.units ),
                               initializer=self.bias_initializer,
                               trainable=True)
      
            
            

    def call(self, inputs):
        """ GC-MC has three inputs : [An, X，Z]
        """
        self.An=inputs[0]
        self.X = inputs[1]
        self.Z = inputs[2]   
        
 
        
        ### 计算 XW1
        
        #print('self.X, self.weight_1',self.X,self.weight_1)
        XW1 = tf.add(tf.matmul(self.X, self.weight_1),self.bias_1) #这一步可以用sparseTensor，不知道有没有必要
        
        #计算 (CA)ZW2
        #1.计算C，不需要了，在数据预处理部分已经计算了CA
        pass
        
        #2.计算ZW2   
        ZW2 = tf.add(tf.matmul(self.Z, self.weight_2),self.bias_2) 
        
        # 计算 (CA)T ZW2

        
        if self.An.shape[-1]!=ZW2.shape[0]:
            CA_T=tf.sparse.transpose(self.An)
            output1= tf.sparse.sparse_dense_matmul(CA_T,ZW2) 
        else:
            output1= tf.sparse.sparse_dense_matmul(self.An,ZW2)  


        
        #计算 XW1+(CA)T ZW2
      
        output=tf.add(XW1,output1)        
               
        if self.activation:
            output = self.activation(output)

        return output

In [6]:
class Self_Attention(tf.keras.layers.Layer):
    def __init__(self,
                 att_size,
                 activation=lambda x: x,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',):
        super(Self_Attention, self).__init__()
        self.activation = activations.get(activation)
        self.att_size=att_size
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        
    def build(self, input_shape):
        
        weight_att_shape_x=input_shape[-1]
        weight_att_shape_y=self.att_size
        
        self.weight_att=self.add_weight(name="weight_to_att",
                               shape=(weight_att_shape_x, weight_att_shape_y),
                               initializer=self.kernel_initializer, 
                               trainable=True)
        
        self.bias=self.add_weight(name="bias",
                               shape=(weight_att_shape_y,),
                               initializer=self.bias_initializer,
                               trainable=True)
        
        self.weight_att_u=self.add_weight(name="weight_to_att_u",
                               shape=(weight_att_shape_y, 1),
                               initializer=self.kernel_initializer,
                               trainable=True)

    
    def call(self, inputs):

        self.X = inputs
        
        WX=tf.matmul(self.X,self.weight_att)
        V=tf.tanh(tf.add(WX,self.bias))
        VU=tf.matmul(V,self.weight_att_u)
        alphas=tf.nn.softmax(VU,axis=0) 
        
        outputs=tf.multiply(alphas,self.X)
        outputs_att=tf.reduce_sum(outputs,axis=0) 
        

        alphas_new=tf.transpose(tf.squeeze(alphas, axis=-1)) #需要改变attention形状

        return outputs_att,alphas_new

In [7]:
def preprocess_graph(adj):
    # _A = A + I
    _adj = adj + sp.eye(adj.shape[0])
    # _dseq：各个节点的度构成的列表
    _dseq = _adj.sum(1).A1
    # 构造开根号的度矩阵
    _D_half = sp.diags(np.power(_dseq, -0.5))
    # 计算标准化的邻接矩阵, @ 表示矩阵乘法
    adj_normalized = _D_half @ _adj @ _D_half
    return adj_normalized.tocsr()

In [8]:
# 稀疏矩阵转稀疏张量
def sp_matrix_to_sp_tensor(M):
    if not isinstance(M, sp.csr.csr_matrix):
        M = M.tocsr()
    # 获取非0元素坐标
    row, col = M.nonzero()
    # SparseTensor参数：二维坐标数组，数据，形状
    X = tf.SparseTensor(np.mat([row, col]).T, M.data, M.shape)
    X = tf.cast(X, tf.float32)
    return X

In [9]:
class Multi_GCN_GCMC_Model(tf.keras.Model):
    def __init__(self,
                 Muilt_AA,
                 Muilt_AP,
                 Muilt_AT,
                 Muilt_AC,
             
                 
                 Muilt_FA,
                 Muilt_FP,
                 Muilt_FT,
                 Muilt_FC,
    
                 output_size=3): 

        # 调用父类__init__()方法
        super(Multi_GCN_GCMC_Model, self).__init__()
        
        self.Muilt_AA=Muilt_AA
        self.Muilt_AP=Muilt_AP
        self.Muilt_AT=Muilt_AT
        self.Muilt_AC=Muilt_AC
        
        
        
        self.Muilt_FA =Muilt_FA
        self.Muilt_FP =Muilt_FP
        self.Muilt_FT =Muilt_FT
        self.Muilt_FC =Muilt_FC
     
        

        
        #同质图
        self.GCN_layer_list_heo_AA=[]
        for i in range(len(self.Muilt_AA)):
            self.GCN_layer_list_heo_AA.append(
                GCNConv(64,activation=tf.keras.activations.relu)
            )


        
        #二部图
#         self.GCMC_layer_list_bio_AP=[]
#         for i in range(len(self.Muilt_AP)):
#             self.GCMC_layer_list_bio_AP.append(
#                 GCMC_Conv(64,activation=tf.keras.activations.relu)
#             )
        
#         self.GCMC_layer_list_bio_AT=[]
#         for i in range(len(self.Muilt_AT)):
#             self.GCMC_layer_list_bio_AT.append(
#                 GCMC_Conv(64,activation=tf.keras.activations.relu)
#             )
        
#         self.GCMC_layer_list_bio_AC=[]
#         for i in range(len(self.Muilt_AC)):
#             self.GCMC_layer_list_bio_AC.append(
#                 GCMC_Conv(64,activation=tf.keras.activations.relu)
#             )
   
 
        
        
        #注意力
        self.att_layer = Self_Attention(32)

    
        #Decoder部分
        self.dense=tf.keras.layers.Dense(output_size)
        




    def call(self,input_x_id, training=False,dropout=0.):
        # 输入数据
        
        #同质网络I 编码过程
   
        _h1_for_A_list=[]
        for i in range(len(self.Muilt_AA)):
            _h1_for_A_list.append(
                self.GCN_layer_list_heo_AA[i]([self.Muilt_AA[i], self.Muilt_FA]) if training \
                else tf.nn.dropout(self.GCN_layer_list_heo_AA[i]([self.Muilt_AA[i], self.Muilt_FA]), dropout)
                
            )

            

#         #二部图AP 编码过程
  
#         _h2_for_A_list=[]
#         for i in range(len(self.Muilt_AP)):
#             _h2_for_A_list.append(
#                 self.GCMC_layer_list_bio_AP[i]([self.Muilt_AP[i], self.Muilt_FA,self.Muilt_FP]) if training \
#                 else tf.nn.dropout(self.GCMC_layer_list_bio_AP[i]([self.Muilt_AP[i], self.Muilt_FA,self.Muilt_FP]), dropout)  
            
#             )

#         #二部图AT 编码过程

#         _h3_for_A_list=[]
#         for i in range(len(self.Muilt_AT)):
#             _h3_for_A_list.append(
#                 self.GCMC_layer_list_bio_AT[i]([self.Muilt_AT[i], self.Muilt_FA,self.Muilt_FT]) if training \
#                 else tf.nn.dropout(self.GCMC_layer_list_bio_AT[i]([self.Muilt_AT[i], self.Muilt_FA,self.Muilt_FT]), dropout)  
            
#             )
            
            
#         #二部图AC 编码过程!!!!!!111111
   
#         _h4_for_A_list=[]
#         for i in range(len(self.Muilt_AC)):
#             _h4_for_A_list.append(
#                 self.GCMC_layer_list_bio_AC[i]([self.Muilt_AC[i], self.Muilt_FA,self.Muilt_FC]) if training \
#                 else tf.nn.dropout(self.GCMC_layer_list_bio_AC[i]([self.Muilt_AC[i], self.Muilt_FA,self.Muilt_FC]), dropout)   
            
#             )
            
            
  
          
        
       

            
        

        #注意力需要最后连接，同质图和二部图的表示向量，

        #_h_for_A_list_set=tf.stack(_h1_for_A_list+_h2_for_A_list+_h3_for_A_list+_h4_for_A_list,axis=0)
        
        _h_for_A_list_set=tf.stack(_h1_for_A_list,axis=0)
        _h7_for_A,att_for_A=self.att_layer(_h_for_A_list_set) 

        #解码层
        output=self.dense(_h7_for_A) 
        
     
        
        
        ###########################################################################
        output_list=[]
        for idx in input_x_id:
            output_list.append(output[idx]) #返回指定id的所有向量
        ########################################################################     

        att=[att_for_A]
        ######################################################################### 
 
        
       
            
        output=[output_list,]
        embedding=_h7_for_A

            
            
        return output,att,embedding

# 加入数据测试一下

In [10]:
#获取文件夹下所有文件名
import os
path=r'C:\Users\Hone\Desktop\Experimental_code\DBLP_2\计算的各个元路径矩阵'
name_list=[]
filelist=os.listdir(path)
for filename in filelist:
    name_list.append(filename)

In [11]:
#取出矩阵的名字
matrix_name_list=[]
for name in name_list:
    matrix_name_list.append(name.split('.')[0].split('_matrix_')[1])

In [12]:
matrix_name_list

['APA',
 'APCP',
 'APCPA',
 'APCPT',
 'APTP',
 'APTPA',
 'APTPC',
 'CPA',
 'CPAPA',
 'PA',
 'PAPA',
 'TPA',
 'TPAPA',
 'TPTPA']

In [13]:
def find_matrix_from_file(path,key_name):
    filelist=os.listdir(path)
    
    for name in filelist:
        if key_name==name.split('.')[0].split('_matrix_')[1]:
            Matrix = sp.load_npz(path+'\\'+name) #读取
            return Matrix
        else:
            pass
        

    print('cant find %s in %s' %key_name %path)
    

In [14]:
#需要将矩阵按照类型进行区分，放进不同的dict
data_AA={}
data_AP={}
data_AT={}
data_AC={}



for pathname in matrix_name_list:
    if pathname[0]==pathname[-1]: #同质邻接矩阵
        data_AA[pathname]=find_matrix_from_file(path,pathname) #输入目标位置文件，与指定文件名，返回该稀疏矩阵
    elif pathname[0]=='P' or pathname[-1]=='P':
        data_AP[pathname]=find_matrix_from_file(path,pathname) 
    elif pathname[0]=='T' or pathname[-1]=='T':
        data_AT[pathname]=find_matrix_from_file(path,pathname) 
    elif pathname[0]=='C' or pathname[-1]=='C':
        data_AC[pathname]=find_matrix_from_file(path,pathname)


In [15]:
Paths_name_AA=[keyword for keyword in data_AA]
Paths_name_AP=[keyword for keyword in data_AP]
Paths_name_AT=[keyword for keyword in data_AT]
Paths_name_AC=[keyword for keyword in data_AC]

In [16]:
def adj_list(name,data_dict):
    matepath_adj_list=[]
    for i in name:
        mat=data_dict[i]
        #矩阵预处理过程，包括DAD,CA
        #########################################
        #如果是方阵，令对角线元素为零, 处理成DAD
        if mat.shape[0]==mat.shape[1]:
            mat=preprocess_graph(mat) #A to DAD
        
        #非方阵，处理成CA  
        else: 
            #统一矩阵形式，N x M, N>=m 
            if mat.shape[0] < mat.shape[1]:
                mat=np.transpose(mat)
                
            #计算C
            sum_row=np.sum(mat,axis=1)
            sum_column=np.sum(mat,axis=0)
            C=sum_row @ sum_column
            
            #C矩阵预处理,
            #C[C==0]=1 
            C=1./np.sqrt(C) 
            C=np.nan_to_num(C,0) #将nan转成0
            
            #CA
            mat=mat.todense()
            mat=np.multiply(C,mat)
            mat=sp.csr_matrix(mat)
        
        
        ##########################################
        matepath_adj_list.append(sp_matrix_to_sp_tensor(mat)) #转tensor.sparse

    return matepath_adj_list

In [17]:
AA_matrix=adj_list(Paths_name_AA,data_AA)
AP_matrix=adj_list(Paths_name_AP,data_AP)
AT_matrix=adj_list(Paths_name_AT,data_AT)
AC_matrix=adj_list(Paths_name_AC,data_AC)

C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in sqrt
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
C:\ProgramData\Miniconda3\envs\TF2.0\lib\site-packa

In [18]:
FA=np.eye(4057)
FP=np.eye(14376)
FT=np.eye(8920)
FC=np.eye(20)

# FA=np.random.rand(4057,128)
# FP=np.random.rand(14376,128)
# FT=np.random.rand(8920,128)
# FC=np.random.rand(20,128)

In [19]:
# #输入训练id集合
import random

index_list_for_FA=[i for i in range(4057)] #需要随机化
random.shuffle(index_list_for_FA) #index乱序
    
a_part=int(len(index_list_for_FA)/10)
train_idx_for_FA = index_list_for_FA[:int(a_part*2)]  # 2 1 7
val_idx_for_FA = index_list_for_FA[ int(a_part*2)+1:int(a_part*3)]
test_idx_for_FA = index_list_for_FA[ int(a_part*3)+1:len(index_list_for_FA)]

In [20]:
# model=Multi_GCN_GCMC_Model(AA_matrix,
#                            AP_matrix,
#                            AT_matrix,
#                            AC_matrix,
                      
                           
#                            FA,
#                            FP,
#                            FT,
#                            FC,
                        
                           
#                            output_size=4
#                           )

In [21]:
# output,att,embedding=model(index_list_for_FA,training=True,dropout=0.)

In [22]:
# model.summary()

In [23]:
#标签
data_label=sp.load_npz(r'C:\Users\Hone\Desktop\Experimental_code\DBLP_2\sparse_matrix_author_label.npz') #读取
data_label=data_label.todense()
data_label=np.array(data_label)

In [24]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.005) #learning_rate=0.001
loss_func=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
#这个loss函数还不够全，还得加入L2正则化

train_loss=tf.keras.metrics.Mean('train_loss') #计算loss的均值
train_acc_I=tf.keras.metrics.CategoricalAccuracy('train_accuracy_I')


test_loss=tf.keras.metrics.Mean('test_loss')
test_acc_I=tf.keras.metrics.CategoricalAccuracy('test_accuracy_I')

In [25]:
def train_step(model,id_x,labels_x,dropout=0.):  #一个set为一个batch的量，
    #tf.GradientTape()记录运算过程的变量梯度
    
    with tf.GradientTape() as t:
        output_list,att,embedding=model(input_x_id=id_x,training=True,dropout=dropout)    #计算一个batch的预测值   #!!!
     
        loss_step_1=loss_func(labels_x,output_list[0])   #计算一个batch的损失函数
        loss_step=loss_step_1
        
 
    grads=t.gradient(loss_step,model.trainable_variables)  #更新变量的梯度
    
    
    #利用跟新的梯度计算变量，model.trainable_variables表示网络中所有的变量
 
    optimizer.apply_gradients(zip(grads,model.trainable_variables))  
    train_loss(loss_step)  #传入计算一个epoch累计loss函数中
    train_acc_I(labels_x,output_list) #传入计算一个epoch累计accuracy函数中
    
    
    return att

In [26]:
def test_step(model,id_x,labels_x): 
    output_list,att,embedding=model(input_x_id=id_x,training=False,dropout=0.) 
    
    loss_step_1=loss_func(labels_x,output_list[0])   #计算一个batch的损失函数
    #loss_step_2=loss_func(labels_x,output_list[1])   
    loss_step=loss_step_1
    
    test_loss(loss_step)
    test_acc_I(labels_x,output_list) #传入计算一个epoch累计accuracy函数中

In [27]:
def split_list_by_n(list_collection, n):
    """
    将集合均分，每份n个元素
    :param list_collection:
    :param n:
    :return:返回的结果为评分后的每份可迭代对象
    """
    for i in range(0, len(list_collection), n):
        yield list_collection[i: i + n]

In [28]:
def train(
          AA_matrix,
          AP_matrix,
          AT_matrix,
          AC_matrix,
       
    
          FA,
          FP,
          FT,
          FC,
     
    
          epoch_time,  
          batch_size,
          
          
          id_x_set, 
          id_x_set_for_val,
          id_x_set_for_test,
          
  
          label_set_I,
          output_size=3,
          dropout=0.
         ): #需要给要训练的集合和验证集合
    
    model=Multi_GCN_GCMC_Model(AA_matrix,
                          AP_matrix,
                          AT_matrix,
                          AC_matrix,
                         
                               
                          FA,
                          FP,
                          FT,
                          FC,
         
                               
                          output_size=output_size
                          )
    

    best_acc_result=0
    for epoch in range(epoch_time):
        
        #training
        batch_idx_train=[] #获取一个batch的数据
        for i in split_list_by_n(id_x_set,batch_size): #split_list_by_n这个函数
            batch_idx_train.append(i)  
                     
        #创建batch_label矩阵

        
        for a_batch_x in batch_idx_train: 
            
            batch_label_list_x=[]

            for id_x in a_batch_x:
                batch_label_list_x.append(label_set_I[id_x]) 
                
    

            att=train_step(model,
                           id_x=a_batch_x,
                           labels_x=batch_label_list_x,
                           dropout=dropout) 
            print('一个train_batch训练完毕')
                           
        print('Epoch {} ，train_loss={:.3f}，train_acc_I={:.3f}'.format(epoch,
                                                                        train_loss.result(),
                                                                        train_acc_I.result())) 
        
        
        
        
        
        #validating
        batch_idx_val=[] #获取一个batch的数据
        
        
        
        for i in split_list_by_n(id_x_set_for_val,batch_size):
            batch_idx_val.append(i)
            

            
        for a_batch_x in batch_idx_val:
            batch_label_list_x=[]
            
            for id_x in a_batch_x:
                batch_label_list_x.append(label_set_I[id_x]) #获取一个batch的所有label

                
            
            test_step(model,id_x=a_batch_x, labels_x=batch_label_list_x)
            print('一个val_batch验证完毕')
            
        print('val_loss={:.3f}，val_accuracy_I={:.3f},best_accuracy={:.3f}'.format(test_loss.result(),test_acc_I.result(),best_acc_result)
             )
        
        ####################################################################################
        #保存结果最佳的embedding
        test_acc=float(test_acc_I.result())
        if test_acc>best_acc_result:
            
            best_acc_result=test_acc
           
            
            
            if best_acc_result>0.8:
                
                #需要返回，train数据载入得到的att
                att=train_step(model,
                    id_x=id_x_set,
                    labels_x=label_set_I[id_x_set]) 
    
                #输出test数据集的embedding 
                #输入test数据集，输出test数据集的embedding
                _,att_test,embedding=model(input_x_id=id_x_set_for_test,
                          training=False) 
                #还需要一个对应的label矩阵
                #从label_set_I中取出对应id的值，重新组成一个新的矩阵
                embedding_list=[]
                new_label_matrix_list=[]
                for i in id_x_set_for_test:
                    new_label_matrix_list.append(label_set_I[i])
                    embedding_list.append(embedding[i])
                    new_label_matrix=np.array(new_label_matrix_list)
    
                #保存embedding和对应的label矩阵
                np.save(r'C:\Users\Hone\Desktop\My_model-DATA\DBLP\embedding_of_test_data_HeMGNN-C-sparse',np.array(embedding_list))
                np.save(r'C:\Users\Hone\Desktop\My_model-DATA\DBLP\label_matrix_of_test_data_HeMGNN-C-sparse',new_label_matrix)
                print('表示向量保存完毕！')

        train_loss.reset_states()
        train_acc_I.reset_states()

        
        test_loss.reset_states()
        test_acc_I.reset_states()
        ####################################################################################
        
        
        
    #循环结束,对test数据验证
    batch_idx_test = []  # 获取一个batch的数据

    
    for i in split_list_by_n(id_x_set_for_test, batch_size):
        batch_idx_test.append(i)

    

    for a_batch_x in batch_idx_test:
        batch_label_list_x = []

        
        for id_x in a_batch_x:
            batch_label_list_x.append(label_set_I[id_x])  # 获取一个batch的所有label
    
        

        test_step(model,id_x=a_batch_x,labels_x=batch_label_list_x)
    print('End of model training\n'
          'test_loss={:.3f}，test_accuracy_I={:.3f}'.format(test_loss.result(),
                                                          test_acc_I.result()))
    

    test_loss.reset_states()
    test_acc_I.reset_states()

          
    return att

In [32]:
att=train(
    AA_matrix=AA_matrix,
    AP_matrix=AP_matrix,
    AT_matrix=AT_matrix,
    AC_matrix=AC_matrix,
 
    
    FA=FA,
    FP=FP,
    FT=FT,
    FC=FC,

   
    epoch_time=20,  
    batch_size=1000,
          
          
    id_x_set          =train_idx_for_FA, 
    id_x_set_for_val  =val_idx_for_FA,
    id_x_set_for_test =test_idx_for_FA,

  
    label_set_I=data_label,
    output_size=data_label.shape[1],
    dropout=0.
)

一个train_batch训练完毕
Epoch 0 ，train_loss=1.386，train_acc_I=0.248
一个val_batch验证完毕
val_loss=1.383，val_accuracy_I=0.292,best_accuracy=0.000
一个train_batch训练完毕
Epoch 1 ，train_loss=1.378，train_acc_I=0.388
一个val_batch验证完毕
val_loss=1.377，val_accuracy_I=0.272,best_accuracy=0.292
一个train_batch训练完毕
Epoch 2 ，train_loss=1.367，train_acc_I=0.356
一个val_batch验证完毕
val_loss=1.370，val_accuracy_I=0.267,best_accuracy=0.292
一个train_batch训练完毕
Epoch 3 ，train_loss=1.351，train_acc_I=0.377
一个val_batch验证完毕
val_loss=1.361，val_accuracy_I=0.275,best_accuracy=0.292
一个train_batch训练完毕
Epoch 4 ，train_loss=1.332，train_acc_I=0.414
一个val_batch验证完毕
val_loss=1.351，val_accuracy_I=0.282,best_accuracy=0.292
一个train_batch训练完毕
Epoch 5 ，train_loss=1.308，train_acc_I=0.478
一个val_batch验证完毕
val_loss=1.340，val_accuracy_I=0.295,best_accuracy=0.292
一个train_batch训练完毕
Epoch 6 ，train_loss=1.280，train_acc_I=0.549
一个val_batch验证完毕
val_loss=1.328，val_accuracy_I=0.307,best_accuracy=0.295
一个train_batch训练完毕
Epoch 7 ，train_loss=1.246，train_acc_I=0.662


In [30]:
att_1=tf.reduce_sum(att[0],axis=0)/att[0].shape[0]
att_weihgt_1=list(np.array(att_1))
for i in att_weihgt_1:
    print('%.3f' %i)

0.396
0.346
0.258
